# Table-Valued Function Edge Cases

This notebook tests edge cases and error conditions for the TVF implementation.

In [ ]:
import duckdb
import pandas as pd

## Edge Case 1: Missing Return Type

Test what happens when no return type is specified (should default to 'records').

In [ ]:
def simple_function():
    return [("default_test", 123)]

conn = duckdb.connect()
schema = [("name", "VARCHAR"), ("value", "INTEGER")]

print("=== Testing Missing Return Type ===")

try:
    # Register without specifying return_type (should default to 'records')
    tvf = conn.create_table_function(
        name="default_return_type",
        callable=simple_function,
        schema=schema
        # Note: no return_type parameter
    )
    
    print("✓ TVF registered successfully with default return type")
    
    # Test that it works (should default to records behavior)
    result = conn.execute("SELECT * FROM default_return_type()").fetchall()
    print(f"✓ Default return type works: {result}")
    
except Exception as e:
    print(f"✗ Error with default return type: {e}")

conn.close()

## Edge Case 2: Invalid Return Type

Test error handling for completely invalid return type values.

In [ ]:
def test_function():
    return [("test", 1)]

conn = duckdb.connect()
schema = [("col", "VARCHAR"), ("val", "INTEGER")]

print("=== Testing Invalid Return Types ===")

invalid_types = [
    "invalid",
    "pandas", 
    "numpy",
    "json",
    "xml",
    "",  # empty string
    "RECORDS",  # wrong case
    "Arrow_Table",  # wrong case
]

for return_type in invalid_types:
    try:
        conn.create_table_function(
            name=f"test_invalid_{len(return_type)}_{hash(return_type) % 1000}",
            callable=test_function,
            schema=schema,
            return_type=return_type
        )
        
        # If registration succeeds, try to use it to trigger bind-time error
        result = conn.execute(f"SELECT * FROM test_invalid_{len(return_type)}_{hash(return_type) % 1000}()").fetchall()
        print(f"✗ '{return_type}' should have failed but succeeded: {result}")
        
    except Exception as e:
        if "Unknown return type" in str(e):
            print(f"✓ '{return_type}' correctly rejected: {e}")
        else:
            print(f"? '{return_type}' failed with unexpected error: {e}")

conn.close()

## Edge Case 3: Invalid Schema Definitions

Test various invalid schema configurations.

In [ ]:
def test_function():
    return [("test", 1)]

conn = duckdb.connect()

print("=== Testing Invalid Schema Definitions ===")

# Test cases with various invalid schemas
invalid_schemas = [
    # Empty schema
    ([], "empty_schema"),
    
    # Schema with invalid type names
    ([("col1", "INVALID_TYPE")], "invalid_type"),
    
    # Schema with missing column name
    ([("col1", "VARCHAR"), ("", "INTEGER")], "empty_column_name"),
    
    # Schema with None values
    ([("col1", "VARCHAR"), (None, "INTEGER")], "none_column_name"),
    
    # Schema with wrong tuple size
    ([("col1",)], "incomplete_tuple"),
    
    # Schema with too many elements in tuple
    ([("col1", "VARCHAR", "extra")], "too_many_elements"),
]

for schema, test_name in invalid_schemas:
    try:
        conn.create_table_function(
            name=f"test_{test_name}",
            callable=test_function,
            schema=schema,
            return_type="records"
        )
        
        # If registration succeeds, try to use it
        result = conn.execute(f"SELECT * FROM test_{test_name}()").fetchall()
        print(f"? {test_name}: Unexpectedly succeeded with result: {result}")
        
    except Exception as e:
        print(f"✓ {test_name}: Correctly failed with: {e}")

conn.close()

## Edge Case 4: Schema-Data Mismatch

Test what happens when the function returns data that doesn't match the schema.

In [ ]:
conn = duckdb.connect()

print("=== Testing Schema-Data Mismatch ===")

# Test 1: Function returns more columns than schema expects
def too_many_columns():
    return [("a", 1, "extra", 999)]  # 4 columns but schema expects 2

schema = [("col1", "VARCHAR"), ("col2", "INTEGER")]

try:
    conn.create_table_function(
        name="too_many_cols",
        callable=too_many_columns,
        schema=schema,
        return_type="records"
    )
    
    result = conn.execute("SELECT * FROM too_many_cols()").fetchall()
    print(f"? Too many columns: Unexpectedly succeeded: {result}")
    
except Exception as e:
    print(f"✓ Too many columns: Correctly failed: {e}")

# Test 2: Function returns fewer columns than schema expects
def too_few_columns():
    return [("only_one",)]  # 1 column but schema expects 2

try:
    conn.create_table_function(
        name="too_few_cols",
        callable=too_few_columns,
        schema=schema,
        return_type="records"
    )
    
    result = conn.execute("SELECT * FROM too_few_cols()").fetchall()
    print(f"? Too few columns: Unexpectedly succeeded: {result}")
    
except Exception as e:
    print(f"✓ Too few columns: Correctly failed: {e}")

# Test 3: Function returns wrong data types
def wrong_types():
    return [(123, "should_be_int")]  # First should be VARCHAR, second INTEGER

try:
    conn.create_table_function(
        name="wrong_types",
        callable=wrong_types,
        schema=schema,
        return_type="records"
    )
    
    result = conn.execute("SELECT * FROM wrong_types()").fetchall()
    print(f"✓ Wrong types: Handled gracefully with type conversion: {result}")
    
except Exception as e:
    print(f"? Wrong types: Failed: {e}")

conn.close()

## Edge Case 5: Non-Iterable Return Values

Test error handling when function returns unexpected data structures.

In [ ]:
conn = duckdb.connect()
schema = [("col", "VARCHAR")]

print("=== Testing Non-Iterable Return Values ===")

# Test different invalid return types
test_cases = [
    ("returns_none", lambda: None, "function returns None"),
    ("returns_string", lambda: "not_a_list", "function returns string"),
    ("returns_int", lambda: 42, "function returns integer"),
    ("returns_dict", lambda: {"key": "value"}, "function returns dictionary"),
    ("empty_list", lambda: [], "function returns empty list"),
]

for func_name, func, description in test_cases:
    try:
        conn.create_table_function(
            name=func_name,
            callable=func,
            schema=schema,
            return_type="records"
        )
        
        result = conn.execute(f"SELECT * FROM {func_name}()").fetchall()
        print(f"? {description}: Unexpectedly succeeded: {result}")
        
    except Exception as e:
        print(f"✓ {description}: Correctly handled: {e}")

conn.close()

## Edge Case 6: Function That Raises Exceptions

Test error handling when the Python function itself raises exceptions.

In [ ]:
conn = duckdb.connect()
schema = [("col", "VARCHAR")]

print("=== Testing Functions That Raise Exceptions ===")

# Test various exception types
def raises_value_error():
    raise ValueError("This is a test error")

def raises_type_error():
    raise TypeError("Type error in function")

def raises_runtime_error():
    raise RuntimeError("Runtime error occurred")

def division_by_zero():
    return [(1/0, "test")]  # This will raise ZeroDivisionError

error_functions = [
    ("value_error_func", raises_value_error),
    ("type_error_func", raises_type_error),
    ("runtime_error_func", raises_runtime_error),
    ("division_error_func", division_by_zero),
]

for func_name, func in error_functions:
    try:
        conn.create_table_function(
            name=func_name,
            callable=func,
            schema=schema,
            return_type="records"
        )
        
        print(f"✓ {func_name}: Registered successfully")
        
        # Try to execute - this should propagate the error
        result = conn.execute(f"SELECT * FROM {func_name}()").fetchall()
        print(f"? {func_name}: Unexpectedly succeeded: {result}")
        
    except Exception as e:
        print(f"✓ {func_name}: Correctly propagated error: {e}")

conn.close()

## Edge Case 7: Deregistering Non-Existent Functions

Test what happens when trying to unregister functions that don't exist.

In [ ]:
conn = duckdb.connect()

print("=== Testing Deregistration of Non-Existent Functions ===")

non_existent_functions = [
    "never_registered",
    "completely_fake",
    "",  # empty string
    "generate_series",  # built-in function name
]

for func_name in non_existent_functions:
    try:
        conn.unregister_table_function(func_name)
        print(f"✓ {func_name or '<empty>'}: Deregistration completed (no error for non-existent)")
        
    except Exception as e:
        print(f"? {func_name or '<empty>'}: Error during deregistration: {e}")

conn.close()

## Edge Case 8: Complex Data Types in Schema

Test various DuckDB data types in schema definitions.

In [ ]:
from decimal import Decimal
from datetime import date, time

def complex_data():
    return [(
        "test_string",
        42,
        3.14159,
        True,
        date(2023, 12, 25),
        Decimal("999.99"),
        b"binary_data"
    )]

conn = duckdb.connect()

print("=== Testing Complex Data Types ===")

complex_schemas = [
    # Basic types
    ([
        ("str_col", "VARCHAR"),
        ("int_col", "INTEGER"),
        ("float_col", "DOUBLE"),
        ("bool_col", "BOOLEAN"),
        ("date_col", "DATE"),
        ("decimal_col", "DECIMAL(10,2)"),
        ("blob_col", "BLOB")
    ], "complex_types"),
    
    # Edge case types
    ([
        ("tiny_int", "TINYINT"),
        ("small_int", "SMALLINT"),
        ("big_int", "BIGINT"),
        ("real_col", "REAL"),
        ("timestamp_col", "TIMESTAMP"),
    ], "edge_types"),
]

for schema, test_name in complex_schemas:
    try:
        # Adjust function to match schema size
        if test_name == "edge_types":
            test_func = lambda: [(1, 2, 3, 4.0, "2023-12-25 10:30:00")]
        else:
            test_func = complex_data
        
        conn.create_table_function(
            name=f"test_{test_name}",
            callable=test_func,
            schema=schema,
            return_type="records"
        )
        
        result = conn.execute(f"SELECT * FROM test_{test_name}()").fetchall()
        print(f"✓ {test_name}: Successfully handled complex types: {result}")
        
    except Exception as e:
        print(f"? {test_name}: Error with complex types: {e}")

conn.close()

## Summary

This notebook tests various edge cases and error conditions:

1. **Missing Return Type**: Handled gracefully with appropriate defaults
2. **Invalid Return Types**: Proper error messages with valid type suggestions
3. **Invalid Schemas**: Various schema validation and error handling
4. **Schema-Data Mismatch**: Column count and type mismatch handling
5. **Non-Iterable Returns**: Error handling for invalid return data structures
6. **Function Exceptions**: Proper propagation of Python exceptions
7. **Non-Existent Deregistration**: Graceful handling of invalid unregister calls
8. **Complex Data Types**: Support for various DuckDB data types in schemas

These tests ensure the TVF implementation is robust and handles edge cases appropriately.